In [19]:
import random
import pymongo
#建立mongo客户端
client=pymongo.MongoClient('mongodb://localhost:27017/')
#清空test01数据库
client.drop_database('test01')
#预创建test01的数据库
db=client.test01
#创建python对象,test01中的集合
coll=db.coll


In [20]:
#插入记录
rs=coll.insert_one({'a':1,"b":2})
object_id=rs.inserted_id
print(rs.inserted_id)

5ac5d09f150337227c307dd4


In [21]:
#插入多条记录
rs=coll.insert_many([{'a':random.randint(1,10),'b':10} for _ in range(10)])

In [22]:
print(rs.inserted_ids)

[ObjectId('5ac5d0a0150337227c307dd5'), ObjectId('5ac5d0a0150337227c307dd6'), ObjectId('5ac5d0a0150337227c307dd7'), ObjectId('5ac5d0a0150337227c307dd8'), ObjectId('5ac5d0a0150337227c307dd9'), ObjectId('5ac5d0a0150337227c307dda'), ObjectId('5ac5d0a0150337227c307ddb'), ObjectId('5ac5d0a0150337227c307ddc'), ObjectId('5ac5d0a0150337227c307ddd'), ObjectId('5ac5d0a0150337227c307dde')]


In [23]:
#查询单条记录
print(coll.find_one({'a':1,'b':2}))

{'_id': ObjectId('5ac5d09f150337227c307dd4'), 'a': 1, 'b': 2}


In [24]:
#查询当前结合全部文档数
print(coll.count())

11


In [26]:
#查询结果是游标
cursor=coll.find({'a':{'$lte':1}})
#符合文档的文档数
print(cursor.count())

3


In [27]:
for r in cursor:
    print(r,r['b'])

{'_id': ObjectId('5ac5d09f150337227c307dd4'), 'a': 1, 'b': 2} 2
{'_id': ObjectId('5ac5d0a0150337227c307dd8'), 'a': 1, 'b': 10} 10
{'_id': ObjectId('5ac5d0a0150337227c307dda'), 'a': 1, 'b': 10} 10


In [28]:
cursor=coll.find()
print(cursor.count())


for r in cursor:
    print(r)

11


In [34]:
#对查询结果排序
# 先find后sort
#find() 括号中是字典  非比较符直接写值,  比较符用字典形式   key是操作符   value是参考值
# sort()  括号中是列表  列表中是元祖   元祖,逗号隔开
print(list(             coll.find({'a':{'$lte':1}}).sort([('b',-1)])         ))

[{'_id': ObjectId('5ac5d0a0150337227c307dd8'), 'a': 1, 'b': 10}, {'_id': ObjectId('5ac5d0a0150337227c307dda'), 'a': 1, 'b': 10}, {'_id': ObjectId('5ac5d09f150337227c307dd4'), 'a': 1, 'b': 2}]


In [41]:
print(coll.find({'b':{'$gt':1}}).limit(1).skip(1).next())
             

{'_id': ObjectId('5ac5d0a0150337227c307dd5'), 'a': 10, 'b': 10}


In [47]:
rs=coll.find_one_and_update({'a':1,'b':2},{'$set':{'b':3},'$inc':{'a':1}},upsert=False)

In [49]:
print(rs )

{'_id': ObjectId('5ac5d09f150337227c307dd4'), 'a': 1, 'b': 2}


In [50]:
print(list(coll.find({'a':2,'b':3})))

[{'_id': ObjectId('5ac5d09f150337227c307dd4'), 'a': 2, 'b': 3}]


In [51]:
coll.find_one_and_update({'a':1,'b':2},{'$set':{'b':3},'$inc':{'a':1}},upsert=True)

In [52]:
print(coll.find({'a':2,'b':3}).count())

2


In [53]:
#删除单个文档
coll.delete_one({'a':2,'b':3})

In [54]:
#删除多个文档
rs=coll.delete_many({'a':2,'b':3})

In [57]:
print(rs.deleted_count)

1


In [59]:
#管道
#通过$match获得符合条件的文档,传递给group来分组
#分组计算总数count和a的平均值averageA,传递给project
#project过滤掉字段_id
cursor=coll.aggregate([
    {'$match':{'b':{'$gt':1,'$lt':11}}},
    {'$group':{'_id':None,'count':{'$sum':1},'averageA':{'$avg':'$a'}}},
    {'$project':{'_id':0,'count':1,'averageA':1}}
])

In [61]:
cursor.next()

{'averageA': 6.2, 'count': 10}

In [62]:
from bson.code import Code

In [124]:
mapper=Code("""
function(){
var key;
if (this.a<3){
key='lt 3';
}else{
key='gte 3';
}
emit(key,1);
}
""")

In [87]:
reducer=Code("""
function(key,values){
return Array.sum(values)
}
""")


In [89]:
#c存放结果的集合, query用来过滤需要计算的文档,
#符合条件的才会做MapReduce计算
rs=coll.map_reduce(mapper,reducer,'c',query={'b':{'$gt':1,'$lt':11}})

In [119]:
for i in rs.find():
    print(i)

{'_id': 'gte 3', 'value': 8.0}
{'_id': 'lt 3', 'value': 2.0}


_id就是上面的键
符合大于或者等于3的文档有8个
符合晓宇3的有2个

{'_id': 'gte 3', 'value': 8.0}
{'_id': 'lt 3', 'value': 2.0}

In [125]:
print(mapper)
coll.insert_one({'id':'mapper','value':mapper})
cursor=coll.find()
print(cursor.count())


for r in cursor:
    print(r)


function(){
var key;
if (this.a<3){
key='lt 3';
}else{
key='gte 3';
}
emit(key,1);
}

12
{'_id': ObjectId('5ac5d0a0150337227c307dd5'), 'a': 10, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dd6'), 'a': 3, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dd7'), 'a': 3, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dd8'), 'a': 1, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dd9'), 'a': 10, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dda'), 'a': 1, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307ddb'), 'a': 7, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307ddc'), 'a': 9, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307ddd'), 'a': 10, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dde'), 'a': 8, 'b': 10}
{'_id': ObjectId('5ac5f0aa150337227c307de7'), 'id': 'mapper', 'value': None}
{'_id': ObjectId('5ac5f0b3150337227c307de8'), 'id': 'mapper', 'value': Code("\nfunction(){\nvar key;\nif (this.a<3){\nkey='lt 3';\n}else{\nkey='gte 3';\n}\nemit(key,1);\n}\n", None)}


In [143]:

mapper=coll.find_one({'id':'mapper','value':{'$ne':None}})['value']
print(mapper)



function(){
var key;
if (this.a<3){
key='lt 3';
}else{
key='gte 3';
}
emit(key,1);
}



In [145]:
print(coll.distinct('a'))


[10, 3, 1, 7, 9, 8]


In [111]:
cursor=coll.find()
print(cursor.count())


for r in cursor:
    print(r)

10
{'_id': ObjectId('5ac5d0a0150337227c307dd5'), 'a': 10, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dd6'), 'a': 3, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dd7'), 'a': 3, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dd8'), 'a': 1, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dd9'), 'a': 10, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dda'), 'a': 1, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307ddb'), 'a': 7, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307ddc'), 'a': 9, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307ddd'), 'a': 10, 'b': 10}
{'_id': ObjectId('5ac5d0a0150337227c307dde'), 'a': 8, 'b': 10}
